# An introduction to Python yield

## WHAT

### A function and a generater

In [ ]:
def fun():
    return 'fun'

def gen():
    yield 'gen'

In [ ]:
fun()

In [ ]:
gen()

### A range() like example


**for** ... **in range()** loop is used quite often when iterating objects in `python`.

In below example, we made a generator to mock **range()** function.

In [34]:
def counter(top):
    n = 0
    while n < top:
        yield n
        n += 1

In [35]:
for i in counter(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


### Under the covers

Generator object runs in response to **next()** or **send()**

In [36]:
def counter_sample(top):
    n = 0
    while n < top:
        print('-> before yield')
        yield n
        print('-> after yield')
        n += 1
        
c = counter_sample(5)

In [37]:
print(next(c))

-> before yield
0


**StopIteration** raised when function returns

In [26]:
print(c.send(None))

-> after yield
-> before yield
5


### Create coroutine with yield

You can **send** data to a coroutine.

In [76]:
def generator():
    item = yield
    print('item = {}'.format(item))
    yield 10

**Prime** the coroutine

In [77]:
g = generator()
g.send(None)

<img src="generator_send_data.PNG">

Send data

In [78]:
value = g.send(20)

item = 20


In [79]:
print('value = {}'.format(value))

value = 10


A coroutine which **receives** data as well as **produces** data

In [ ]:
def averager():
    total, count, average = 0.0, 0, None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [ ]:
avg = averager()
avg.send(None)

In [ ]:
avg.send(10)

In [ ]:
avg.send(20)

In [ ]:
avg.send(30)

**Three features of coroutines:**
- When a coroutine run into **yield**, it will suspend
- A **caller** should **schedule** the coroutine when it suspended
- When a coroutine suspended, it will **return control** to the **caller**

## WHY

### A tornado example

In [ ]:
import time
import tornado.ioloop
import tornado.web
import tornado.gen


class BadStupidHandler(tornado.web.RequestHandler):
    def get(self):
        for i in range(20):
            self.write('{}<br>'.format(i))
            self.flush()
            time.sleep(0.5)


class GoodStupidHandler(tornado.web.RequestHandler):
    @tornado.gen.coroutine
    def get(self):
        for i in range(20):
            self.write('{}<br>'.format(i))
            self.flush()
            yield tornado.gen.sleep(0.5)

app = tornado.web.Application([
     (r'/bad', BadStupidHandler),
     (r'/good', GoodStupidHandler)
])
app.listen(8001)
tornado.ioloop.IOLoop.current().start()


Refer to documentation [Frequently Asked Questions](http://www.tornadoweb.org/en/stable/faq.html).

## HOW

### Use coroutine to simplfy your context manager

A context manager is to change:

In [ ]:
try:
    f = open('some.txt')
    print(f.readline()) # do something with f
finally:
    f.close()

To:

In [ ]:
with open('some.txt') as f:
    print(f.readline()) # do something with f

You can define your own context manager with a **class** implements `__enter__` and `__exit__` method

In [ ]:
class Mirror:
    def __init__(self, num):
        self.num = num
    
    def __enter__(self):
        import sys
        
        
        def reverse_write(text):
            self.original_write(text[::-1])
        
        self.original_write = sys.stdout.write
        sys.stdout.write = reverse_write
        return 'This is mirror {}'.format(self.num)
        
    def __exit__(self,exc_type, exc_value, traceback):
        import sys
        
        sys.stdout.write = self.original_write

In [ ]:
with Mirror(1000) as first_string:
    print(first_string)
    print(123456789)
print('Out there')

Use decorator `contextlib.contextmanager` and `generator` to simplfy your own context manager

In [ ]:
import contextlib

@contextlib.contextmanager
def Mirror_new(num):
    import sys
    

    def reverse_write(text):
        original_write(text[::-1])
    
    original_write = sys.stdout.write
    sys.stdout.write = reverse_write
    yield 'This is mirror {}'.format(num)
    sys.stdout.write = original_write

In [ ]:
with Mirror_new(1000) as first_string:
    print(first_string)
    print(123456789)
print('Out there')

How does it work?

We define a warpper class to proxy our generator:

In [ ]:
class  GeneratorCM:        
    def __init__(self, func):
        self._func = func
        
    def __call__(self, *args, **kwargs):
        self._gen = self._func(*args, **kwargs)
        return self
        
    def __enter__(self):
        return self._gen.send(None)
    
    def __exit__(self,exc_type, exc_value, traceback):
        try:
            self._gen.send(None)
        except StopIteration:
            return True

In [ ]:
@GeneratorCM
def Mirror_custom(num):
    import sys
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write
    yield 'This is mirror {}'.format(num)
    sys.stdout.write = original_write

In [ ]:
with Mirror_custom(1000) as first_string:
    print(first_string)
    print(123456789)
print('Out there')

GeneratorCM with full try catch is listed below:

In [ ]:
class  GeneratorCM:        
    def __init__(self, func):
        self._func = func
        
    def __call__(self, *args, **kwargs):
        self._gen = self._func(*args, **kwargs)
        return self
        
    def __enter__(self):
        return self._gen.send(None)
    
    def __exit__(self,exc_type, exc_value, traceback):
        try:
            if exc_type is None:
                next(self._gen)
            else:
                self._gen.throw(exc_type, exc_value, traceback)
                raise RuntimeError("Generator didn't stop")
        except StopIteration:
                return True
        except:
                if sys.exc_info()[1] is not exc_value: raise

### Inlined yield

Following statement is very common in tornado framwork

In [ ]:
from tornado import gen

@gen.coroutine
def fetch_coroutine(url):
    http_client = AsyncHTTPClient()
    response = yield http_client.fetch(url)
    raise gen.Return(response.body)

How does it work?

First,define a slow function to mock `http.fetch` :

In [ ]:
import time
import random

def func(x, y):    
    sleep_time = random.random() * 3
    time.sleep(sleep_time)
    print('sleep for {} seconds'.format(sleep_time))
    return x + y

We need to our function work like this:

In [ ]:
from concurrent.futures import ThreadPoolExecutor, Future
pool = ThreadPoolExecutor(max_workers=8)

Inspired by @contextmanager：

<img src="inline_yield.PNG">

In [ ]:
import wrapt

class Task:
    def __init__(self, gen):
        self._gen = gen
        initive = Future()
        initive.set_result(None)
        self.step(initive)

    def step(self, future):
        try:
            next_future = self._gen.send(future.result())
        except StopIteration as exc:
            if exc.value is not None:
                raise exc
        else:
            next_future.add_done_callback(self.step)
        
@wrapt.decorator
def inlined_future(wrapped, instance, args, kwargs):
    Task(wrapped(*args,**kwargs))

In [ ]:
@inlined_future
def do_func(x, y):
    result = yield pool.submit(func, x, y)
    print('Got:', result)
    
def do_func_slow(x,y):
    result=func(x , y)
    print('Got:', result)

In [ ]:
for i in range(5):
    do_func_slow(i,i)

In [ ]:
for i in range(5):
    do_func(i,i)

## Additional: How coroutine work

In [ ]:
import dis
import inspect

def gen_fn():
    result = yield 1
    print('result of yield: {}'.format(result))
    result2 = yield 2
    print('result of 2nd yield: {}'.format(result2))
    return 'done'

def normal_fn():
    return 1

In [ ]:
a = gen_fn()

In [ ]:
a.send(None)

In [ ]:
gen_fn

In [ ]:
normal_fn

In [ ]:
bool(gen_fn.__code__.co_flags & inspect.CO_GENERATOR)

In [ ]:
bin(inspect.CO_GENERATOR)

In [ ]:
bool(normal_fn.__code__.co_flags & inspect.CO_GENERATOR)

In [ ]:
gen1 = gen_fn()

type(gen1)

In [ ]:
gen1.gi_code.co_name

<img src="generator_object.PNG">

All generators from calls to gen_fn point to this same code. But each has its own stack frame. This stack frame is not on any actual stack, it sits in heap memory.

In [ ]:
gen2 = gen_fn()

gen1.gi_code is gen2.gi_code

In [ ]:
gen1.gi_frame is gen2.gi_frame

In [ ]:
gen1.send(None)

gen1.gi_frame.f_lasti

In [ ]:
dis.dis(gen1)

In [ ]:
gen1.send('hello')

gen1.gi_frame.f_lasti

In [ ]:
gen1.gi_frame.f_locals

In [ ]:
gen1.send('world')

## Reference

1. [Generators: The Final Frontier](http://www.dabeaz.com/finalgenerator/)
2. [Effective Python：Consider Coroutines to Run Many Functions Concurrently](http://www.effectivepython.com/2015/03/10/consider-coroutines-to-run-many-functions-concurrently/)
3. [Python Cookbook 3: 不用递归实现访问者模式](http://python3-cookbook.readthedocs.io/zh_CN/latest/c08/p22_implementing_visitor_pattern_without_recursion.html)
4. [500 line or less:A Web Crawler With asyncio Coroutines](http://aosabook.org/en/500L/a-web-crawler-with-asyncio-coroutines.html)
5. [一个简单的文本解析计算器](https://motor-taxi-master-rider.github.io/python/2017/08/31/a-simple-text-complie-calculator-with-coroutine-and-other-tricks)